## Preparation

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [1]:
import tqdm
import os
import cv2
import imghdr
import image_to_numpy
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime
import itertools

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomContrast, RandomBrightness, Rescaling, Resizing
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, ReLU
from tensorflow.keras.metrics import F1Score, Precision, Recall, CategoricalAccuracy

import sklearn
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import StratifiedKFold

In [2]:
data_dir = 'D:\DATASETS\For-Thesis\Labels'

In [3]:
def sample_count(data_dir):
    for dir in os.listdir(data_dir):
        print("{}: {}".format(dir, len(os.listdir(os.path.join(data_dir,dir)))))

In [4]:
sample_count(data_dir)

Building: 600
Plant: 600
Road: 600
Vehicle: 600


## Prepare Dataset

In [5]:
labels = ['Building', 'Plant', 'Road', 'Vehicle']
X_train = []
y_train = []
y_label_idx = []
image_size = 256
for i in labels:
    folderPath = os.path.join('D:\DATASETS\For-Thesis\Labels', i)
    for j in os.listdir(folderPath):
        img = image_to_numpy.load_image_file(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_label_idx.append(labels.index(i))

In [6]:
X_train = np.array(X_train)
y_label_idx = np.array(y_label_idx)

# Use these for training
X_train_scaled = X_train.astype(np.float32) / 255.0
y_one_hot = tf.keras.utils.to_categorical(y_label_idx)

## Prepare Functions for KFold

In [7]:
def make_model():
    model = Sequential()
    model.add(RandomFlip(mode='horizontal', seed=1, input_shape=(256,256,3)))
    model.add(RandomRotation(factor=0.2, seed=1))

    model.add(Conv2D(input_shape=(256,256,3), filters=16, kernel_size=(3,3), padding="same"))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(MaxPooling2D())

    model.add(Conv2D(filters=16, kernel_size=(3,3), padding="same"))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(MaxPooling2D())

    model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same"))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(MaxPooling2D())

    model.add(Flatten())
    model.add(Dropout(0.2, seed=1))
    model.add(Dense(units=128,activation="relu"))
    model.add(Dense(units=64,activation="relu"))

    model.add(Dense(units=4, activation="softmax"))
    model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
def create_callbacks():
    file_name = 'Model_V4-0-1_KFOLD_' + datetime.now().strftime("%Y%m%d-%H%M%S")
    log_dir = f'./logs/{file_name}'
    checkpoint_filepath = f'./checkpoints/{file_name}'

    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, 
                                                 histogram_freq=1,
                                                 profile_batch=2,
                                                 write_graph=True,
                                                 write_images=True,
                                                 update_freq='epoch',
                                                 embeddings_freq=1
                                                )
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, 
                                                             monitor='val_accuracy', 
                                                             mode='auto', 
                                                             save_best_only=True, 
                                                             verbose=1)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                     factor=0.3, 
                                                     patience=2, 
                                                     min_delta=0.001, 
                                                     mode='auto', 
                                                     verbose=1)
    callbacks = [tb_callback, cp_callback, reduce_lr]
    return callbacks

In [9]:
def plot_loss(hist):
    fig = plt.figure(figsize=(4,3))
    plt.plot(hist.history['loss'], color='teal', label='loss')
    plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
    plt.suptitle('Loss')
    plt.legend(loc='upper right')
    plt.show()

In [10]:
def plot_acc(hist):
    fig = plt.figure(figsize=(4,3))
    plt.plot(hist.history['accuracy'], color='teal', label='acc')
    plt.plot(hist.history['val_accuracy'], color='orange', label='val_acc')
    plt.suptitle('Accuracy')
    plt.legend(loc='upper left')
    plt.show()

In [12]:
kfold = StratifiedKFold(n_splits=6, shuffle=True, random_state=1)

In [13]:
for train, test in kfold.split(X_train_scaled, y_one_hot):
    model = make_model()
    hist = model.fit(X_train_scaled[train], y_one_hot[train],
                     batch_size=8,
                     epochs=20,
                     verbose=1,
                     callbacks=create_callbacks())
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    plot_loss(hist)
    plot_acc(hist)
    print(f'Categorical Accuracy: {scores}')

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multilabel-indicator' instead.